Python提供了`__future__`模块，把下一个新版本的特性导入到当前版本，于是我们就可以在当前版本中测试一些新版本的特性。  
`re`是正则表达式模块  
`torch.optim`是为了更方便的更新  
这个网址 https://blog.csdn.net/stupid_3/article/details/83184691 和B站李宏毅的课程讲的比较清楚

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
print(device)

cuda


In [3]:
SOS_token = 0
EOS_token = 1
NNNN_token = 2#这个是用于补足句子的


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS": 0, "EOS": 1, "NNNN": 2}
        self.word2count = {"SOS": 0, "EOS": 0, "NNNN": 0}
        self.index2word = {0: "SOS", 1: "EOS", 2: "NNNN"}
        self.n_words = 3  # Count SOS and EOS and NNNN

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10
MAX_LEN = 12

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def standardizeSeq(s):
    return "SOS " + s + " EOS" + " NNNN" * (MAX_LEN - len(s.split(' ')) - 2)

def filterPairs(pairs):
    #这个是前面不补SOS，后面不补EOS，也不补助的情况。
    #return [pair for pair in pairs if filterPair(pair)]
    #这个是补了SOS，EOS，且长度补齐的情况
    return [(standardizeSeq(pair[0]), standardizeSeq(pair[1])) for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4346
eng 2804
('SOS je suis sur de son nom . EOS NNNN NNNN NNNN', 'SOS i m sure about his name . EOS NNNN NNNN NNNN')


各个变量的维度：以下为singal head,现在代码实现了multihead,代码中不一样的已标出（这个和下面的矩阵横竖是一样的，第一维是横，一般输入也是第一维是长度嘛，然后竖着表示特征数）  
input: len \* input_size(特征数）  
output: len \* input_size(特征数)  
matq: len \* hidden_size  
matv: 同上  
matk: 同上  
scale:  那个就是指 $\sqrt{output\_size}$ ，除以就是减小结果  
然后是我的代码中的:  
attention: 与论文中的不一样，就是q,v产生的，len \* len  
mattv: 这个是论文中的attention, len \* hidden_size  
y: len \* input_size
首先我们要明白self-attention是什么，其实它输入输出和RNN是一样的，就是用于替换RNN层  
各个变量的现实意义：  
个人理解：q表示询问，可以理解当前这个元素的信息，而k则是key，可以理解为询问对这个问题产生的关注是多少，将这个元素和某个元素组合，得到attention，v则是值，表示当前元素的值，用于经过这样的attention,最后产生的影响
![avatar](self-attention1.jpg)
然后我们可以把向量叠成matrix来简化运算,其中q,k,v都是input通过矩阵乘法产生的
![avatar](matrix1.jpg) ![avatar](matrix2.jpg)
首先理解scaledDotProductAttention是在干什么：  
![avatar](scaled_dot-product_attention.jpg)  
scaledDotAttention就是在干这个，之所以可以写成矩阵乘法就是如同上面的那个图，下面贴一个完整的图：
![avatar](sdpa.jpg)
其中的那个scale就是前面说的除以减小结果的意思  
而mask就是为了不再不必要的地方产生attention，比如为了使句子长度相同而后补的0，比如在decoder时输出时不应看到未来的，例如现在在句中的位置t，
就应仅与句子中的t-1前的内容产生attention。而mask的方法就是在那些位置填-inf,这样softmax后就变成0了。下面的`tensor.mask_fill_(bool_matrix,value)`  
下面是multiheadAttention  
![avatar](mha.jpg)
concat就是把几个head拼接起来，h是head的数量

注意linear是这样瞎搞的，它的输入也是第一维是长度，和前面是对应的，中间随你放多少维，反正最后一维是特征数
![avatar](linear.jpg)

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_size, hidden_size, head = 2):
        super(MultiHeadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.hidden_dim = hidden_size // head
        self.input_size = input_size
        self.head = head
        
        self.qlinear = nn.Linear(input_size,hidden_size)
        self.vlinear = nn.Linear(input_size,hidden_size)
        self.klinear = nn.Linear(input_size,hidden_size)
        self.ylinear = nn.Linear(hidden_size,input_size)
        
    def scaledDotProductAttention(self, matq, matk, matv, mask):
        scale = matk.size(-1)**0.5
        matt = matq.matmul(matk.transpose(1,2))/scale  #(head, N, hidden_dim) * (head, hidden_dim, N) -> (head, N, N)
        if not mask is None:
            matt.masked_fill_(mask, -np.inf)
        matt = F.softmax(matt, dim=-1)  #(head, N, N) * (head, N, hidde_dim)
        mattv = matt.matmul(matv)
        return torch.cat(torch.chunk(mattv.view(-1,self.hidden_dim), self.head,0), dim = 1) #(head, N, hidden_dim) ->(N, hidden_size)
    
    def toMulti(self, matx):#这里是把特征等分
        return torch.cat(torch.chunk(matx, self.head, -1),dim=0).view(self.head, -1, self.hidden_dim)
    
    def forward(self, inputq, inputk, inputv, mask = None):
        matq = self.toMulti(self.qlinear(inputq))
        matv = self.toMulti(self.vlinear(inputv))
        matk = self.toMulti(self.klinear(inputk))
        mattv = self.scaledDotProductAttention(matq, matk, matv, mask)
        y = self.ylinear(mattv)
        return y

In [9]:
def sequece_mask(input):
    len = input.size(-2)
    return torch.triu(torch.ones((len,len), dtype = torch.bool), diagonal = 1).to(device)
    #里面ones和zeros差不多，就是产生1； dtype之所以用这个是因为mask需要为bool
    #而triu函数则是将参数一的上三角部分返回回去，剩下的补0，而diagonal决定了是否多返回或少返回，看下面这张图

![avatar](triu.jpg)

In [10]:
#开始检查没有mask的对不对
model1 = MultiHeadAttention(5, 2).to(device)
input1 = torch.randn(6,5).to(device)
y1 = model1(input1,input1,input1)
print(y1)
torch.sum(y1).backward()

tensor([[ 0.3875, -0.2006,  0.3170, -0.2971,  0.1688],
        [ 0.2606, -0.0670,  0.4240, -0.3522,  0.1500],
        [ 0.3206, -0.1465,  0.3349, -0.4020,  0.1780],
        [ 0.2632, -0.0559,  0.4544, -0.2865,  0.1342],
        [ 0.3246, -0.1183,  0.4079, -0.2498,  0.1408],
        [ 0.3193, -0.1351,  0.3595, -0.3562,  0.1661]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [11]:
#开始检查有mask的对不对
model1 = MultiHeadAttention(5, 2).to(device)
input1 = torch.randn(6,5).to(device)
y1 = model1(input1,input1,input1,torch.tensor([[False,False,False,False,False,True]]*6).to(device))
print(y1)
torch.sum(y1).backward()

tensor([[-0.2618,  0.5105, -0.5245, -0.1616, -0.6298],
        [-0.2770,  0.5536, -0.5358, -0.2473, -0.6445],
        [-0.2678,  0.5127, -0.5555, -0.1879, -0.6279],
        [-0.2697,  0.5786, -0.4471, -0.2286, -0.6614],
        [-0.2618,  0.5474, -0.4578, -0.1801, -0.6491],
        [-0.2392,  0.5413, -0.3367, -0.0820, -0.6574]], device='cuda:0',
       grad_fn=<AddmmBackward>)


位置编码，讲解见后，这里仅给出公式以及这个公式为什么可行：
$$PE_{(pos,2*i)} = sin(pos/{10000^{2i/d_{model}}})$$
$$PE_{(pos,2*i+1)} = cos(pos/{10000^{2i/d_{model}}})$$
这所以可行，是因为这不仅可以表示绝对位置，也可以表示相对位置，$sin(x+y)=sin(x)cos(y)+cos(x)sin(y)$，这意味着$PE(pos+k)$可以用$PE(pos)$和$PE(k)$来表示，故可以表示相对位置

In [ ]:
class PositionEncoding(nn.Module):
    def __init__(self, max_len, word_size):
        super(PositionEncoding, self).__init__()
        self.position_mat = torch.tensor([[pos/np.power(10000,i//2*2/word_size) for i in range(word_size)] for pos in range(max_len)], \
                                        dtype = torch.float32).to(device)
        self.position_mat[:,0::2] = torch.sin(self.position_mat[:,0::2])
        self.position_mat[:,1::2] = torch.cos(self.position_mat[:,1::2])
    
    def forward(self, input):
        return input + self.position_mat[0:input.size(-2),0:input.size(-1)]

![avatar](transformer.jpg)
embedded: len \* word_size 这里面需要考虑位置的影响，不然之前那样不考虑位置是不科学的，这个就是有一种奇特的方法加上一个向量。为什么是加呢？
可以这样理解：把原来的input延长，补一个位置one-hot变量，那么乘一个矩阵压到这么多维，可以相等于加一个矩阵。embedding的格式
就是把input的每个元素都扩展成word_size维的向量，它的两个重要参数第一个是字典大小，第二个是word_size。后面那个参数padding_idx=0表示遇到这个数时生成全为0的向量，比如你输入为了补齐，补了一些0(这份代码补的是2)，这些啥用都没有，就直接生成全0就好了
output: len \* word_size  
output2: len \* word_size  
图中每一个的意义： 
n:首先这个是进行了多次，原论文是6次，本次本着性能不足的原因，就进行一次，若要多次，可把这个封装成一个类，在encodertran调多次即可。  
add:就是input+output，加的目的是使梯度>1，防止梯度消失   
norm:就是layernormalization  
feed forward:就是一个最普通前馈式神经网络，为什么要接这个我也不明白。网上有大佬说接这个是为了使更有效（expressiveness，不知道咋翻译），理由是
前面的都是线性的，而这个可以带来了一些非线性的混合。

normalization归一化：使数据的均值和方差变成指定的数，方便学习。
![avatar](layernorm.jpg)
那个E是均值的意思，Var是方差的意思，$\gamma$是新的方差，$\beta$是新的均值  
layerNormalization就是把一个数据自身的所有特征进行normalization，而batchNormalization就是把一个batch的所有数据的同一个特征拿来normalization  
至于为什么要有$\gamma$$\beta$，网上大佬的解释是使他具有capacity，比如如果你后接了一个ReLU,没这个的话一半的输入都输出0，就太不合理了。

In [ ]:
class EncoderTran(nn.Module):
    #word_number是指字典大小，word_size是embedding的产生出来的size,hidden_size就是multiheadAttention中的hidden_size
    #ff_size就是那个前馈式神经网络的隐藏层，max_len就是句子的最长长度，是用来处理位置向量的
    def __init__(self, word_number, word_size = 256, hidden_size = 380, ff_size = 400, max_len = MAX_LEN):
        super(EncoderTran, self).__init__()
        self.word_size = word_size
        self.hidden_size = hidden_size
        self.ff_size = ff_size
        self.max_len = max_len
        
        self.embedding = nn.Embedding(word_number, word_size, padding_idx = 2)
        self.position_mat = PositionEncoding(max_len, word_size)
        self.multi_head_attention = MultiHeadAttention(word_size,hidden_size)
        self.layernorm1 = nn.LayerNorm(word_size)#里面直接填特征数就可以了，输入len * 特征数就可以了
        self.ffn_linear1 = nn.Linear(word_size, ff_size)
        self.ffn_linear2 = nn.Linear(ff_size,word_size)
        self.layernorm2 = nn.LayerNorm(word_size)
    
    def forward(self, input):
        embedded =  self.position_mat(self.embedding(input))
        #这个mask会自动广播，可以结合图看，只需要使得attention矩阵涉及0的竖行为0就可以了，
        #因为横行一是不影响，aij才表示j位置对i位置的attention,只要使添0的地方对有内容的不影响就行了
        #二是反正softmax前都全为0，即使写出了-inf一是一样的
        output = self.multi_head_attention(embedded, embedded, embedded, input.eq(2))
        output = self.layernorm1(output + embedded)
        output2 = self.ffn_linear2(F.relu(self.ffn_linear1(output)))
        output2 = self.layernorm2(output2+output)
        return output2

In [ ]:
#开始检查对不对
model1 = EncoderTran(10, 5, 4, 3, 9).to(device)#小细节，测试的时候不要让维度有相同的数，这样可以检查维度正不正确
input1 = torch.tensor([1,2,7,8,0,9,7,2]).to(device)
y1 = model1(input1)
print(y1)
torch.sum(y1).backward()

注意：此处，我们让输出和输入长度在训练时都是相同的，方便以后改成可以不一个一个输入，一个batch一个batch输入的情况，在第二个attention(即encoder-decoder attention）那里，注意维度，下面用1指代input，2用来output指代翻译出来的句子。  
matq: len2 \* word_size  
matk: len1 \* word_size  
matv: len1 \* word_size  
attention: len2 \* len1  
output2: len2 \* word_size  
此处已经不需要mask了

In [ ]:
class DecoderTran(nn.Module):
    def __init__(self,  word_number, word_size = 256, hidden_size = 380, ff_size = 400, max_len = MAX_LEN):
        super(DecoderTran, self).__init__()
        self.word_size = word_size
        self.hidden_size = hidden_size
        self.ff_size = ff_size
        self.max_len = max_len
        
        self.embedding = nn.Embedding(word_number, word_size, padding_idx = 2)
        self.position_mat = PositionEncoding(max_len, word_size)
        
        self.multi_head_attention1 = MultiHeadAttention(word_size,hidden_size)
        self.layernorm1 = nn.LayerNorm(word_size)
        
        self.multi_head_attention2 = MultiHeadAttention(word_size,hidden_size)#调用这个的时候确实padding_mask想不清楚，不管了
        self.layernorm2 = nn.LayerNorm(word_size)
        
        self.ffn_linear1 = nn.Linear(word_size, ff_size)
        self.ffn_linear2 = nn.Linear(ff_size,word_size)
        self.layernorm2 = nn.LayerNorm(word_size)
        
        self.ans_linear = nn.Linear(word_size,word_number)
    
    def forward(self, input, enc_output):
        embedded =  self.position_mat(self.embedding(input))
        output = self.multi_head_attention1(embedded, embedded, embedded, sequece_mask(embedded))
        output = self.layernorm1(output + embedded)
        
        output2 = self.multi_head_attention2(output, enc_output, enc_output)#注意顺序
        output2 = self.layernorm2(output2 + output)
        
        output3 = self.ffn_linear2(F.relu(self.ffn_linear1(output2)))
        output3 = self.layernorm2(output3+output2)
        
        output4 = F.log_softmax(self.ans_linear(output3),dim=1)
        return output4

In [136]:
#开始检查对不对
model1 = EncoderTran(10, 5, 4, 3, 9).to(device)#小细节，测试的时候不要让维度有相同的数，这样可以检查维度正不正确
input1 = torch.tensor([1,2,7,8,3,9,7,2,0]).to(device)
y1 = model1(input1)

model2 = DecoderTran(10, 5, 4, 3, 9).to(device)
input2 = torch.tensor([3,5,7,8,3]).to(device)
y2 = model2(input2, y1)
print(y2)
torch.sum(y1).backward()

tensor([[-2.9119, -2.8814, -3.4928, -1.8626, -2.3670, -2.1591, -2.3321, -1.3360,
         -2.2288, -3.6094],
        [-3.2929, -3.6101, -3.3831, -1.6523, -1.9148, -2.6003, -1.9952, -2.3864,
         -1.3948, -4.3591],
        [-2.7500, -3.2198, -2.9127, -1.8429, -1.7206, -3.0195, -2.1177, -2.9308,
         -1.3646, -3.6314],
        [-3.0787, -3.5555, -3.1884, -1.8419, -1.7493, -2.6862, -1.9707, -2.9525,
         -1.2892, -4.1007],
        [-2.9769, -3.1763, -4.1061, -1.3027, -2.4569, -2.2237, -2.3894, -1.7355,
         -1.9452, -4.2553]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)


In [137]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    #indexes.append(EOS_token),前面添加了
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1)#这里把它变成了一个向量

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [138]:
#测试：
print(input_lang.word2index["NNNN"])
pair=random.choice(pairs)
print(pair)
print(tensorsFromPair(pair))
pair=random.choice(pairs)
print(pair)
print(tensorsFromPair(pair))

2
('SOS je ne suis pas comme toi . EOS NNNN NNNN NNNN', 'SOS i am not like you . EOS NNNN NNNN NNNN NNNN')
(tensor([   0,    7,  298,   12,  247, 1287,  118,    6,    1,    2,    2,    2],
       device='cuda:0'), tensor([  0,   3,  17, 148, 744, 130,   5,   1,   2,   2,   2,   2],
       device='cuda:0'))
('SOS nous sommes juste la . EOS NNNN NNNN NNNN NNNN NNNN', 'SOS we re right here . EOS NNNN NNNN NNNN NNNN NNNN')
(tensor([  0, 124, 127, 382, 203,   6,   1,   2,   2,   2,   2,   2],
       device='cuda:0'), tensor([ 0, 78, 79, 68, 47,  5,  1,  2,  2,  2,  2,  2], device='cuda:0'))


In [139]:
#这个函数每次是在拿一个句子进行训练，句子在之前已经补足
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

    #清空梯度
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = encoder(input_tensor)
    
    decoder_outputs = decoder(target_tensor, encoder_outputs)
    loss = criterion(decoder_outputs[0:-1], target_tensor[1:])
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length

In [140]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m,s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [141]:
#这里才是总的训练
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.005):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0].to(device)
        target_tensor = training_pair[1].to(device)
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))
        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    
    showPlot(plot_losses)

In [142]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [143]:
print(device)
encoder1 = EncoderTran(input_lang.n_words).to(device)
decoder1 = DecoderTran(output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 20000, print_every=1000)

cuda
0m 26s (- 8m 16s) (1000 5%) 0.2264
0m 52s (- 7m 54s) (2000 10%) 0.1849
1m 19s (- 7m 30s) (3000 15%) 0.1703
1m 46s (- 7m 4s) (4000 20%) 0.1556
2m 13s (- 6m 40s) (5000 25%) 0.1520
2m 39s (- 6m 12s) (6000 30%) 0.1463
3m 5s (- 5m 45s) (7000 35%) 0.1427
3m 32s (- 5m 19s) (8000 40%) 0.1370
3m 58s (- 4m 51s) (9000 45%) 0.1397
4m 24s (- 4m 24s) (10000 50%) 0.1384
4m 52s (- 3m 59s) (11000 55%) 0.1347
5m 18s (- 3m 32s) (12000 60%) 0.1312
5m 44s (- 3m 5s) (13000 65%) 0.1307
6m 12s (- 2m 39s) (14000 70%) 0.1255
6m 38s (- 2m 12s) (15000 75%) 0.1272
7m 5s (- 1m 46s) (16000 80%) 0.1208
7m 31s (- 1m 19s) (17000 85%) 0.1222
7m 58s (- 0m 53s) (18000 90%) 0.1210
8m 25s (- 0m 26s) (19000 95%) 0.1173
8m 51s (- 0m 0s) (20000 100%) 0.1186


In [93]:
trainIters(encoder1, decoder1, 10000, print_every=1000)

0m 19s (- 2m 54s) (1000 10%) 0.0904
0m 41s (- 2m 47s) (2000 20%) 0.0887
1m 3s (- 2m 27s) (3000 30%) 0.0929
1m 24s (- 2m 7s) (4000 40%) 0.0884
1m 45s (- 1m 45s) (5000 50%) 0.0878
2m 6s (- 1m 24s) (6000 60%) 0.0839
2m 28s (- 1m 3s) (7000 70%) 0.0830
2m 49s (- 0m 42s) (8000 80%) 0.0795
3m 11s (- 0m 21s) (9000 90%) 0.0766
3m 33s (- 0m 0s) (10000 100%) 0.0790


In [109]:
def evaluate(encoder, decoder, sentence, max_length = MAX_LEN):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        
        encoder_outputs = encoder(input_tensor)
    
        decoder_input = torch.tensor([0], device = device)
        
        decoded_words = []
        
        for di in range(max_length):
            decoder_output = decoder(decoder_input, encoder_outputs)
            topv, topi = decoder_output[di].data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            
            decoder_input = torch.cat((decoder_input, torch.tensor([topi.item()], device = device)))
            
        return decoded_words

In [110]:
def evaluateRandomly(encoder, decoder, n=1):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [147]:
evaluateRandomly(encoder1, decoder1)

> SOS je suis meilleur que lui . EOS NNNN NNNN NNNN NNNN
= SOS i m better than him . EOS NNNN NNNN NNNN NNNN
< i m sorry i m you . <EOS>

